In [1]:
import numpy as np 
import pandas as pd

In [2]:
#task1
scores = pd.read_csv('data_safe_copy.csv')
print(scores.shape,'\n',scores.head())
scores.info()

(300, 5) 
   student_id cohort   module assignment  score
0       S001  alpha  Module1         A1     78
1       S001  alpha  Module1         A2     84
2       S001  alpha  Module2         A1     79
3       S001  alpha  Module2         A2     86
4       S001  alpha  Module3         A1     81
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   student_id  300 non-null    object
 1   cohort      300 non-null    object
 2   module      300 non-null    object
 3   assignment  300 non-null    object
 4   score       300 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 11.8+ KB


In [3]:
#task2
roster = pd.DataFrame({
    'Student ID': ['S051', 'S052', 'S053', 'S054', 'S055', 'S056', 'S057', 'S058', 'S059', 'S060'],
    'status': ['active','inactive','active','inactive','active','inactive','active','inactive','active','inactive']
}).set_index('Student ID')
roster

mergedscores = scores.merge(roster, left_index=True, right_index=True,how='left')
print(mergedscores.shape,'\n',mergedscores.head())
mergedscores['status'].isna().sum()

(300, 6) 
   student_id cohort   module assignment  score status
0       S001  alpha  Module1         A1     78    NaN
1       S001  alpha  Module1         A2     84    NaN
2       S001  alpha  Module2         A1     79    NaN
3       S001  alpha  Module2         A2     86    NaN
4       S001  alpha  Module3         A1     81    NaN


np.int64(300)

In [5]:
#task3 
ch_avg = mergedscores.groupby('cohort').agg({'score':'mean'})
md_avg = mergedscores.groupby('module').agg({'score':'mean'})
print(ch_avg,'\n\n',md_avg)

avg_by_module = ch_avg.merge(md_avg, left_index=True, right_index=True, how='outer', suffixes=('_cohort','_module'))
print(avg_by_module)

            score
cohort           
alpha   78.715686
beta    79.029412
gamma   76.843750 

          score
module        
Module1  77.64
Module2  77.78
Module3  79.25
         score_cohort  score_module
Module1           NaN         77.64
Module2           NaN         77.78
Module3           NaN         79.25
alpha       78.715686           NaN
beta        79.029412           NaN
gamma       76.843750           NaN


In [ ]:
#task3
print("average by module is greater than average by cohort?: ",avg_by_module.iloc[3:, 0].mean()<avg_by_module.iloc[:3, 1].mean())

average by module is greater than average by cohort?: True


In [ ]:
#task4
student_module_report = mergedscores.pivot_table(index='student_id', columns='module', values='score', aggfunc='mean')
print("is number of rows in report equals to unique students?: ", len(scores['student_id'].unique()) == len(student_module_report))
print("label check: ", set(student_module_report.columns) == set(scores['module'].unique()))

is number of rows in report equals to unique students?:  True
label check:  True


In [40]:
#task5
student_averages = scores.groupby(['student_id', 'cohort'])['score'].mean().reset_index()

student_averages = student_averages.rename(columns={'score': 'avg_score'})

top_students = (
    student_averages
    .sort_values(by=['cohort', 'avg_score'], ascending=[True, False])
    .groupby('cohort')
    .head(3)
    .reset_index(drop=True)
)

top_students = top_students[['student_id', 'cohort', 'avg_score']]

cohort_counts = top_students['cohort'].value_counts()

validation_passed = (cohort_counts == 3).all()

print(f"Validation passed (exactly 3 students per cohort): {validation_passed}")
print("\nTop Students Report:")
print(top_students)

Validation passed (exactly 3 students per cohort): True

Top Students Report:
  student_id cohort  avg_score
0       S003  alpha       88.5
1       S049  alpha       88.5
2       S027  alpha       86.5
3       S045   beta       90.5
4       S007   beta       89.5
5       S021   beta       88.5
6       S014  gamma       92.5
7       S040  gamma       84.5
8       S016  gamma       84.0
